### Importing Libraries

In [1]:
from py2neo import Graph
import pandas as pd

### Integrating with Neo4j using py2neo

In [2]:
graph = Graph('bolt://localhost:7687', auth=('neo4j', 'nextproject'))

In [3]:
query ="""
   match(n) return(n)
"""
graph.run(query)

n
"(_0:Encounter {id: '20220001', operation_code: 'CAGM', operation_date: datetime('2019-10-11T00:00:00.000000000+00:00'), patient_id: 'XYZ00001005', procedure_weight: 4})"
"(_1:Encounter {id: '20220002', operation_code: 'CAGM', operation_date: datetime('2019-06-07T00:00:00.000000000+00:00'), patient_id: 'XYZ00005126', procedure_weight: 5})"
"(_2:Encounter {id: '20220003', operation_code: 'CAGM', operation_date: datetime('2019-05-22T00:00:00.000000000+00:00'), patient_id: 'XYZ00005235', procedure_weight: 4})"


In [4]:
# Function for running queries
def run(query):
    return(graph.run(query).stats())

In [5]:
def index_run(query_list):
    for i in query_list:
        graph.run(i).stats()

In [6]:
# query1 =["CREATE INDEX ON :Encounter(id)",
#          "CREATE INDEX ON :Patient (id)",         
#          "CREATE INDEX ON :Condition (code)",  
#          "CREATE INDEX ON :Procedure (code)"]

# index_run(query1)

In [7]:
graph.query("call db.schema.visualization")

nodes,relationships
"[(_-3:Condition {constraints: [], indexes: ['code'], name: 'Condition'}), (_-1:Encounter {constraints: [], indexes: ['id'], name: 'Encounter'}), (_-2:Patient {constraints: [], indexes: ['id'], name: 'Patient'}), (_-4:Procedure {constraints: [], indexes: ['code'], name: 'Procedure'})]","[(_-2)-[:HAS_ENCOUNTER {}]->(_-1), (_-1)-[:HAS_PROCEDURE {}]->(_-4), (_-1)-[:NEXT {}]->(_-1), (_-1)-[:HAS_CONDITION {}]->(_-3), (_-4)-[:SIMILAR {}]->(_-1), (_-4)-[:SIMILAR {}]->(_-4), (_-4)-[:SIMILAR {}]->(_-2)]"


In [ ]:
# query = 'DROP INDEX ON : Condition (code)'
# graph.run(query)

## Loading CSV Files


Loading Encounter.csv and setting the properties of each encounter with its date and Patient date.

In [63]:
query2 = '''
        load csv with headers from "file:/csv/Encounter.csv" as row
        MERGE (e:Encounter {id:row.ENCOUNTER_ID})
          SET 
            e.operation_date=date(row.OPER_DT),
            e.patient_id=row.PATIENT_ID

        MERGE (p:Patient {id:row.PATIENT_ID})
        
        MERGE (p)-[:HAS_ENCOUNTER]->(e)
        
        '''
run(query2)

{'labels_added': 15501,
 'relationships_created': 8694,
 'nodes_created': 15501,
 'properties_set': 32889}

Loading Patient.csv file,creating the node for each patient and adding the properties of the patient.

In [64]:
query3 = '''
      load csv with headers from "file:/csv/Patients.csv" as row
      MERGE (p:Patient {id:row.PATIENT_ID})
          SET
               p.Name=row.PATIENT_NAME, 
               p.gender=row.GENDER,            
               p.weight=row.WEIGHT,
               p.height=row.HEIGHT

        '''
run(query3)

{'properties_set': 21917}

Loading Condition.csv file,creating the node for each and adding the properties of the patient.

In [65]:
query4 = '''
      load csv with headers from "file:/csv/Condition.csv" as row
      MATCH (p:Patient {id:row.PATIENT_ID})
        MERGE (c:Condition {code:row.ENCOUNTER_ID})
        SET 
            c.description=row.DIAGNOSIS_ICD_TEXT,
            c.diagnosis = row.DIAGNOSIS,
            c.operation_date = datetime(row.OPER_DT)
        
        MERGE (cs:Encounter {id:row.ENCOUNTER_ID})
        ON CREATE
          SET cs.date=datetime(row.OPER_DT) 
          
        MERGE (p)-[:HAS_ENCOUNTER]->(cs)
        MERGE (cs)-[:HAS_CONDITION]->(c)

        '''
run(query4)

{'labels_added': 8694,
 'relationships_created': 8694,
 'nodes_created': 8694,
 'properties_set': 28224}

Loading Procedures.csv file,creating the node for each and adding the properties of the patient.

In [66]:
query2 = '''
      load csv with headers from "file:/csv/Procedures.csv" as row
      MERGE (p:Patient {id:row.PATIENT_ID})
      MERGE (r:Procedure {code:row.OPERCODE})
          
          SET 
              r.description=row.OPER_NAME,
              r.operation_date=datetime(row.OPER_DT),
              r.cost = row.COST
          MERGE (pe:Encounter {id:row.ENCOUNTER_ID})
          
          ON MATCH
            SET pe.operation_date=datetime(row.OPER_DT), pe.operation_code=row.OPERCODE

          MERGE (p)-[:HAS_ENCOUNTER]->(pe)
          MERGE (pe)-[:HAS_PROCEDURE]->(r)
        '''
run(query2)

{'labels_added': 367,
 'relationships_created': 8694,
 'nodes_created': 367,
 'properties_set': 43837}

Creating a NEXT relationship 

In [67]:
query2 = '''

    CALL apoc.periodic.iterate(
    'MATCH (p:Patient) RETURN p',
    'MATCH (p)-[:HAS_ENCOUNTER]->(e)
    WITH e
    ORDER BY e.operation_date
    WITH collect(e) AS encounters
    WITH encounters, encounters[1..] as nextEncounters
    UNWIND range(0,size(nextEncounters)-1,1) as index
    WITH encounters[index] as first, nextEncounters[index] as second
    CREATE (first)-[:NEXT{days:duration.inDays(date(first.operation_date), date(second.operation_date))}]->(second)',{iterateList:false});
    
    '''
run(query2)

{}

In [69]:
query2 = '''
       CALL apoc.periodic.iterate('MATCH (c:Condition) RETURN c',
      'MATCH (c)<-[:HAS_CONDITION]-(:Encounter)<-[:HAS_ENCOUNTER]-(p:Patient)
      WITH c,count(p) AS NUM
      SET c.num=NUM', {}
      );
    '''
run(query2)

{}

# Insights

#### Query to get patients and procedures

In [14]:
# 

query = '''
    MATCH (p:Patient)-[:HAS_ENCOUNTER]->(e:Encounter)-[:HAS_PROCEDURE]->(pr:Procedure)
    return p.id as Patient_id,p.Name as Patient_name,collect(pr.code) as sequence
'''
df3 = pd.DataFrame(graph.query(query).to_data_frame())
df3

,Patient_id,Patient_name,sequence
0,ABC00579733,RENJITH R PAI,"[CAGM, EPS, RFA]"
1,ABC00564991,AJAY SAH,"[CAGM, PAGM, PTAAB]"
2,ABC00338680,NIRANJAN DEY,"[CAGM, A203]"
3,ABC00591287,RUDRARAJU ADITYA VARMA,"[CAGM, ICDI]"
4,ABC00587272,S MAIMUNNISA BEGUM,"[CAGM, PTCAP]"
...,...,...,...
6802,ABC00579426,BHOLANATH DHAL,[UROPER7]
6803,ABC00583567,KABITA SARKAR,[UROPER17]
6804,ABC00589884,MST MURSHIDA KHATUN,[UROPER17]
6805,ABC00598605,SURESH HALDAR,[UROPER4]


In [16]:
# # Getting patients who followed CAGM, PTCAP

# df3['sequence'] = df3['sequence'].astype(str)
# df_new1= df3[df3['sequence'] == "['CAGM', 'P1703']"]
# df_new1


#### Query to get patients and procedures

In [ ]:

query = '''
    MATCH (p:Patient)-[:HAS_ENCOUNTER]->(e:Encounter)-[:HAS_PROCEDURE]->(pr:Procedure{code:"AVBDL"})
 match (e)-[:NEXT]->(e1:Encounter)-[:HAS_PROCEDURE]->(pr1:Procedure{code:"AVBDL"})
 return p.id,p.Name,pr.code,pr1.code,count(*)
'''
df1 = pd.DataFrame(graph.query(query).to_data_frame())
df1

#### Counting no of nodes in our graph

In [ ]:
query1 = """
Match(n) return count(n) as ct
"""
result = graph.query(query1)
print(result)

####  Returning Patients names who followed CAGM after P1703

In [ ]:

params = {'p1':'CAGM',
          'p2':'PTCAP'}

query2 = """
        match(n:Patient)-[:HAS_ENCOUNTER]->(e:Encounter{operation_code:$p1})-[:NEXT]->(e1:Encounter{operation_code:$p2})
        //where e1.operation_code CONTAINS $contains 
        return n.id as id
         """

result_df1 = pd.DataFrame(graph.query(query2,parameters = params).to_data_frame())
len(result_df1.id.unique())

#### Counting indegress for different procedures 
#### Also we can know here that which procedure is followed by most of the patients.

In [10]:
query3 = """
            match(p:Procedure)
            return distinct p.code, SIZE(()-[]->(p)) AS inDegree ORDER BY inDegree DESC
         """
result1 = graph.query(query3).to_data_frame()
result1


,p.code,inDegree
0,CAGM,933
1,PTCAP,924
2,P103,335
3,EPS,296
4,PTMCM,288
...,...,...
362,ORTHOPER14,1
363,UROPER23,1
364,UROPER12,1
365,UROPER4,1


## Assosciation rule mining


#### Query to get patients and procedures

In [ ]:
query = '''
    MATCH (p:Patient)-[:HAS_ENCOUNTER]->(e:Encounter)-[:HAS_PROCEDURE]->(pr:Procedure)
    return p.id as Patient_id,collect(pr.code) as sequence
'''
df3 = pd.DataFrame(graph.query(query).to_data_frame())
df3

# Support and Confidence 

In [21]:
query = """
            MATCH (p:Patient)-[:HAS_ENCOUNTER]->(e:Encounter)-[:HAS_PROCEDURE]->(pr:Procedure)
            return p.id as Patient_id,collect(pr.code) as sequence
        """
df4 = pd.DataFrame(graph.query(query).to_data_frame())

In [22]:
df4

,Patient_id,sequence
0,ABC00579733,"[CAGM, EPS, RFA]"
1,ABC00564991,"[CAGM, PAGM, PTAAB]"
2,ABC00338680,"[CAGM, A203]"
3,ABC00591287,"[CAGM, ICDI]"
4,ABC00587272,"[CAGM, PTCAP]"
...,...,...
6802,ABC00579426,[UROPER7]
6803,ABC00583567,[UROPER17]
6804,ABC00589884,[UROPER17]
6805,ABC00598605,[UROPER4]


### Unique procedures and their Count

In [19]:
query = '''
        MATCH (p:Patient)-[:HAS_ENCOUNTER]->(e:Encounter)
        WITH p,e
        ORDER BY e.operation_date
        with p.id as Patient_Id, p.Name as Patient_Name,collect(e.operation_code) as sequence
        return sequence, count(Patient_Id) as count
        order by count(Patient_Id) descending
'''
df7 = pd.DataFrame(graph.query(query).to_data_frame())
df7

In [19]:
query = """
                match (c:Patient)
                CALL{
                        with c 
                        return count(c) as mycount
                }
                with sum(mycount)as count1
                MATCH (p:Patient)-[:HAS_ENCOUNTER]->(e:Encounter)
                WITH p,e
                ORDER BY e.operation_date
                with p.id as Patient_Id, p.Name as Patient_Name,collect(e.operation_code) as sequence
                with toInteger(count(distinct sequence)) as Total_unique,Patient_Id,sequence
                with sequence ,round(toFloat(count(Patient_Id))/895,5) as support
                order by support descending
                return sequence, support 
        """

df6 = pd.DataFrame(graph.query(query).to_data_frame())
df6